In [12]:
import pandas as pd
import numpy as np
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination, ApproxInference

In [2]:
data = pd.read_csv('simulated_data.csv')

In [3]:
TOTAL_DATA_COUNT = len(data)
TOTAL_DATA_COUNT

1000

In [4]:
data.head()

,Age,Risk Factors,COVID19 Status,Cough,Loss of Taste or Smell,Tested Result
0,2,0,2,1,0,1
1,2,0,1,0,1,1
2,1,0,1,0,1,0
3,2,0,0,0,1,2
4,2,0,3,0,0,1


In [5]:
model = BayesianNetwork(
    [('Age', 'COVID19 Status'), 
     ('Risk Factors', 'COVID19 Status'), 
     ('COVID19 Status', 'Loss of Taste or Smell'), 
     ('COVID19 Status', 'Cough'), 
     ('COVID19 Status', 'Tested Result')]
)

model.fit(data)
model.get_cpds()

[<TabularCPD representing P(Age:3) at 0x1f29389a820>,
 <TabularCPD representing P(COVID19 Status:4 | Age:3, Risk Factors:3) at 0x1f29389a9d0>,
 <TabularCPD representing P(Risk Factors:3) at 0x1f29389a250>,
 <TabularCPD representing P(Loss of Taste or Smell:2 | COVID19 Status:4) at 0x1f29389a370>,
 <TabularCPD representing P(Cough:2 | COVID19 Status:4) at 0x1f29389d280>,
 <TabularCPD representing P(Tested Result:3 | COVID19 Status:4) at 0x1f29389d1f0>]

In [13]:
infer1 = VariableElimination(model)
infer2 = ApproxInference(model)

In [7]:
print(infer1.query(variables=['Risk Factors'], evidence={"Loss of Taste or Smell": 1, "Cough":0}))

+-----------------+---------------------+
| Risk Factors    |   phi(Risk Factors) |
+=================+=====================+
| Risk Factors(0) |              0.5870 |
+-----------------+---------------------+
| Risk Factors(1) |              0.3025 |
+-----------------+---------------------+
| Risk Factors(2) |              0.1105 |
+-----------------+---------------------+


In [14]:
print(infer2.query(variables=['Risk Factors'], n_samples=100000, 
                   evidence={"Loss of Taste or Smell": 1, "Cough":0}))

100%|██████████| 100000/100000 [00:00<00:00, 126512.53it/s]

+-----------------+---------------------+
| Risk Factors    |   phi(Risk Factors) |
+=================+=====================+
| Risk Factors(2) |              0.1094 |
+-----------------+---------------------+
| Risk Factors(0) |              0.5884 |
+-----------------+---------------------+
| Risk Factors(1) |              0.3022 |
+-----------------+---------------------+


In [8]:
print(infer1.map_query(variables=['COVID19 Status', 
                                 'Risk Factors', 
                                 'Loss of Taste or Smell', 
                                 'Tested Result'], evidence={"Age": 1, "Cough":1}))

Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]

{'Tested Result': 1, 'Loss of Taste or Smell': 0, 'COVID19 Status': 2, 'Risk Factors': 0}
